# 🎛️ Week 5-6, Notebook 5: The Art and Science of Prompt Engineering

**Module:** LLMs, Prompt Engineering & RAG  
**Project:** Build the Knowledge Core for the Manufacturing Copilot

---

If a Large Language Model is a powerful engine, then a **prompt** is the steering wheel, accelerator, and brake. **Prompt Engineering** is the art and science of designing effective inputs to guide an LLM toward a desired output. It is one of the most critical skills for building reliable, effective, and safe AI applications.

For our Manufacturing Copilot, good prompt engineering is the difference between a helpful assistant and a confusing, unreliable tool. In this notebook, you will learn the fundamental patterns and best practices for crafting prompts that produce accurate, safe, and factually grounded responses for maintenance engineers, operators, and plant managers.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:

1.  **Master Core Prompting Patterns:** Implement and explain the use cases for Zero-Shot, Few-Shot, and Chain-of-Thought (CoT) prompting.
2.  **Enforce Structured Outputs:** Craft prompts that constrain the LLM's output, forcing it to generate specific formats like JSON or follow a predefined schema.
3.  **Assign Roles and Personas:** Use role-based prompts to imbue the model with a specific expertise or personality, improving the quality and relevance of its responses.
4.  **Build a Reusable Prompt Library:** Develop a collection of version-controlled prompt templates for common manufacturing tasks, promoting consistency and maintainability.
5.  **Evaluate Prompts Systematically:** Design and implement a basic evaluation loop to score, compare, and iterate on different prompt variations to find the most effective one.

## ⚙️ Setup: Choosing a Model for Prompt Engineering

While massive, proprietary models like GPT-4 are incredibly powerful, smaller, open-source instruction-tuned models are often more than sufficient for many tasks and offer significant advantages in terms of speed, cost, and privacy.

For this notebook, we will use `google/flan-t5-base`. This is a versatile and widely-used **sequence-to-sequence** model that is particularly good at following instructions and responding to prompts. It is small enough to run on a standard CPU or a modest GPU, making it perfect for experimentation and prototyping.

**A Note on Temperature:** We will set the `temperature` parameter to a low value (e.g., `0.1`). Temperature controls the randomness of the model's output.
*   **High Temperature (e.g., > 0.7):** More creative, random, and diverse outputs. Good for brainstorming or creative writing.
*   **Low Temperature (e.g., < 0.3):** More deterministic, focused, and predictable outputs. Good for factual, instruction-following tasks.

# Hands-On: Setting up the Generation Pipeline
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Set device to GPU if available, otherwise CPU
device = 0 if torch.cuda.is_available() else -1
model_name = "google/flan-t5-base"

# T5 is a sequence-to-sequence model, so we use AutoModelForSeq2SeqLM
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Create the text-generation pipeline
    generator = pipeline(
        'text2text-generation', # Use 'text2text-generation' for T5 models
        model=model,
        tokenizer=tokenizer,
        device=device,
        max_length=256,
        temperature=0.1, # Low temperature for more factual and deterministic outputs
    )

    print(f"Pipeline created successfully for model '{model_name}' on device {device}.")

except Exception as e:
    print(f"Failed to create pipeline. This can happen if the model is not available or if there's a network issue.")
    print(f"Error: {e}")
    # Assign a dummy generator to allow the rest of the notebook to run without errors.
    generator = lambda x: [{"generated_text": "Error: Pipeline not initialized."}]

## 🧪 Pattern 1: Zero-Shot Prompting (The Direct Instruction)

**Zero-shot prompting** is the simplest and most common technique. You simply ask the model to perform a task directly, without providing any prior examples. This relies on the model's vast pre-training knowledge to understand and execute the instruction.

**When to Use:**
*   For simple, straightforward tasks like summarization, translation, or answering general questions.
*   As a baseline to see how well a model performs before moving to more complex techniques.

**Scenario:** A maintenance ticket comes in. We want the LLM to quickly summarize it for a busy shift supervisor.

# Hands-On: Zero-Shot Summarization
# The raw text from an incident report
incident_report = """
During the night shift, the primary coolant pump for CNC-12 (Asset ID: PMP-045) failed, causing a line stoppage.
Telemetry data shows a sudden pressure drop from 60 PSI to 5 PSI over a 30-second period before the automated system shutdown.
The backup pump was engaged manually by the operator, and production resumed after a 15-minute delay.
"""

# A simple, direct prompt that clearly states the task and the input.
zero_shot_prompt = f"""
Summarize the following incident report in one single, concise sentence:

{incident_report}
"""

summary = generator(zero_shot_prompt)

print("--- Zero-Shot Summary ---")
print(summary[0]['generated_text'])

## 🧪 Pattern 2: Few-Shot Prompting (Learning by Example)

**Few-shot prompting** is used when the task is more complex or requires the output to be in a very specific format. By providing a few high-quality examples (the "shots"), you give the model a template to follow. This is a form of **in-context learning**.

**When to Use:**
*   When zero-shot prompting fails or produces inconsistent results.
*   For structured data extraction (e.g., pulling specific fields from a text).
*   To guide the model on how to handle domain-specific jargon.

**Scenario:** We need to extract structured data (Asset, Component, Failure Mode) from unstructured maintenance logs. A zero-shot prompt might struggle with this, but with a couple of examples, the model can learn the desired pattern.

In [ ]:
# Hands-On: Few-Shot Structured Data Extraction
few_shot_prompt = f"""
Extract the Asset, Component, and Failure Mode from the log.

Log: "The vision system on Line-5 is failing to detect part #A55-Z. It seems like a camera calibration issue."
Asset: Line-5
Component: Vision System Camera
Failure Mode: Miscalibration
===
Log: "Press-2 is showing hydraulic pressure fluctuations. The main pump seal is likely worn out and needs replacement."
Asset: Press-2
Component: Hydraulic Pump Seal
Failure Mode: Wear and Tear
===
Log: "{incident_report}"
Asset:"""
# Note: We end the prompt with "Asset:" to guide the model to start generating the answer in the correct format.

structured_data = generator(few_shot_prompt)

print("--- Few-Shot Structured Data Extraction ---")
# The model will continue the pattern. We prepend "Asset:" to the output for a clean result.
print("Asset: " + structured_data[0]['generated_text'])

## 🧪 Pattern 3: Chain-of-Thought (CoT) and Role Prompting

**Chain-of-Thought (CoT) prompting** is a more advanced technique used for complex reasoning tasks. By explicitly instructing the model to "think step-by-step," you encourage it to break down the problem into smaller, more manageable parts. This often leads to more accurate and logical conclusions.

Combining CoT with **role prompting**—where you assign the model a persona or expertise (e.g., "You are a senior reliability engineer")—can make it even more powerful.

**When to Use:**
*   For multi-step reasoning, root cause analysis, or problem-solving.
*   When you need the model to explain its reasoning process.
*   To imbue the model with the knowledge and perspective of a specific expert.

**Scenario:** A complex failure has occurred. We need a preliminary root cause analysis and a recommended action. We will assign the model the role of a "senior reliability engineer" and ask it to follow a specific analytical process.

In [ ]:
# Hands-On: Chain-of-Thought Root Cause Analysis
cot_prompt = f"""
You are a senior reliability engineer with 20 years of experience in industrial maintenance.
Analyze the following incident report by thinking step-by-step.

Report: "{incident_report}"

Follow these steps:
1.  **Sequence of Events:** Detail the events in chronological order.
2.  **Symptoms:** State the primary and secondary symptoms observed.
3.  **Root Cause Hypothesis:** Propose the most likely root cause of the failure.
4.  **Recommended Action:** Recommend the single most important immediate action for the maintenance team.

Begin your analysis:
"""

analysis = generator(cot_prompt)

print("--- Chain-of-Thought Analysis from 'Senior Reliability Engineer' ---")
print(analysis[0]['generated_text'])

## 🗂️ Best Practice: Building a Reusable Prompt Template Library

Hardcoding prompts directly into your application code is inefficient, hard to maintain, and makes it difficult to track changes. A much better approach is to create a centralized library of **prompt templates**.

This approach has several advantages:
*   **Consistency:** Ensures that the same task always uses the same approved prompt.
*   **Maintainability:** If you need to update a prompt, you only have to change it in one place.
*   **Versioning:** You can version your prompts just like you version your code, allowing you to track experiments and roll back to previous versions if needed.
*   **Clarity:** Separates the logic of your application from the specifics of the prompts.

In [ ]:
# Hands-On: Creating and Using a Prompt Template Library
# In a real application, this might be a separate .py or .json file.
PROMPT_TEMPLATES = {
    "v1.summarize_incident": "Summarize this incident report for a shift supervisor in 3 brief bullet points:\n\n{report}",

    "v1.extract_entities_json": """
You are a data extraction bot. From the following text, extract the asset ID, component, and a brief description of the failure.
Respond ONLY with a valid JSON object with the keys 'asset_id', 'component', and 'failure_description'.

Text: "{report}"

JSON:
""",

    "v1.draft_safety_alert": """
You are an EHS (Environment, Health, and Safety) officer. Based on the incident below, draft a 2-sentence safety alert to be posted on the factory floor.
The tone should be urgent, clear, and direct.

Incident: "{report}"

Alert:
"""
}

# --- Example of using a template to enforce JSON output ---
# This is a powerful technique for getting structured data from an LLM.
json_prompt = PROMPT_TEMPLATES['v1.extract_entities_json'].format(report=incident_report)
json_output = generator(json_prompt)

print("--- Prompt Template for JSON Output ---")
print("Prompt Sent to Model:\n", json_prompt)
print("\nGenerated JSON String:\n", json_output[0]['generated_text'])

# You can then parse this string into a Python dictionary
import json
try:
    parsed_json = json.loads(json_output[0]['generated_text'])
    print("\nSuccessfully parsed JSON:")
    print(parsed_json)
except json.JSONDecodeError as e:
    print(f"\nFailed to parse JSON: {e}")

## 📊 Best Practice: Evaluating Prompts Systematically

How do you know if `prompt_v2` is actually better than `prompt_v1`? You must **test and measure**. A systematic evaluation framework is crucial for developing high-quality, reliable prompts. You should never assume a prompt is good just because it "looks right" on a few examples.

**A Simple Evaluation Framework:**

1.  **Define Your Criteria:** What does "better" mean for your use case?
2.  **Create a "Golden Dataset":** A small, high-quality dataset of inputs and their corresponding ideal outputs.
3.  **Run Experiments:** Run different prompt variations against your golden dataset.
4.  **Score the Results:** Score each output against the ideal output based on your criteria.
5.  **Log Everything:** Keep a log of your experiments to track progress and justify your choices.

| Evaluation Criterion | How to Measure | Example |
| :--- | :--- | :--- |
| **Accuracy / Correctness** | Compare model output to a "golden" answer. Can be automated (e.g., `exact match`, `F1 score`) or done by a human reviewer. | Does the `extract_entities_json` prompt correctly identify the asset ID 95% of the time across 100 test cases? |
| **Safety & Compliance** | Use a checklist, a keyword scanner, or even another LLM to flag any outputs that are unsafe, non-compliant, or toxic. | Does the `draft_safety_alert` prompt ever suggest an action that violates company safety policy? |
| **Tone & Style** | Use semantic similarity to compare the output's embedding to a style guide example, or rely on human review. | Is the tone of the safety alert appropriately urgent and professional? |
| **Latency & Cost** | Measure the average response time (latency) and the number of input/output tokens (cost). | Does the complex Chain-of-Thought prompt take too long to be useful in a real-time user interface? |

# Hands-On: A Simple Prompt Evaluation Log
import pandas as pd

# Example of an experiment tracking log you might maintain in a spreadsheet or database.
# This log tracks the performance of different versions of your prompts.
prompt_experiments_log = pd.DataFrame([
    {
        "prompt_name": "summarize_v1_zero_shot",
        "accuracy": 0.75,  # % of summaries deemed 'good' by a human reviewer
        "safety_flags": 0,
        "avg_latency_ms": 450,
        "notes": "Works okay, but sometimes too verbose."
    },
    {
        "prompt_name": "summarize_v2_with_role",
        "accuracy": 0.88,
        "safety_flags": 0,
        "avg_latency_ms": 510,
        "notes": "Better tone and more concise. Adopted as new standard."
    },
    {
        "prompt_name": "extract_json_v1_few_shot",
        "accuracy": 0.96,  # % of fields correctly extracted across the test set
        "safety_flags": 0,
        "avg_latency_ms": 820,
        "notes": "Very accurate but slightly slow. Acceptable for offline processing."
    },
])

print("--- Prompt Evaluation Log ---")
prompt_experiments_log

## ✅ Summary and Next Steps

You now have a powerful and systematic toolkit for guiding and controlling Large Language Models. The key takeaway is to be methodical and data-driven in your approach to prompt design.

**Your Prompt Engineering Workflow:**

1.  **Start Simple:** Always begin with a clear, direct zero-shot prompt. It's often surprisingly effective.
2.  **Iterate and Add Complexity:** If the simple prompt fails, incrementally add more sophisticated techniques. Provide examples (few-shot), assign an expert role, or ask the model to think step-by-step (Chain-of-Thought).
3.  **Enforce Structure:** Don't leave the output format to chance. Explicitly request the structure you need, such as JSON, bullet points, or a specific number of sentences.
4.  **Test, Measure, and Log Everything:** Never assume a prompt is good. Evaluate it against your predefined criteria using a golden dataset and log the results to track your progress.

In the next notebooks, we will combine these advanced prompting techniques with the concept of **Retrieval-Augmented Generation (RAG)**. This will allow us to build a powerful and trustworthy system that can answer questions based on your own private documents and knowledge bases, finally unlocking the full potential of the Manufacturing Copilot.